## A notebook that walks through Surprise and Book Ratings Data

In [1]:
import pandas as pd

read_ = pd.read_csv('books_ratings.csv')

read_.head()

,Unnamed: 0,asin,reviewerID,overall
0,0,000100039X,A2S166WSCFIFP5,5.0
1,1,000100039X,A1BM81XB4QHOA3,5.0
2,2,000100039X,A1MOSTXNIO5MPJ,5.0
3,3,000100039X,A2XQ5LZHTD4AFT,5.0
4,4,000100039X,A3V1MKC2BVWY48,5.0


In [2]:
read_.drop(columns = ['Unnamed: 0'], inplace=True)


In [3]:
read_ = read_[['reviewerID', 'asin', 'overall']]

In [4]:
read_.head()

,reviewerID,asin,overall
0,A2S166WSCFIFP5,000100039X,5.0
1,A1BM81XB4QHOA3,000100039X,5.0
2,A1MOSTXNIO5MPJ,000100039X,5.0
3,A2XQ5LZHTD4AFT,000100039X,5.0
4,A3V1MKC2BVWY48,000100039X,5.0


In [8]:
# Surprise sometimes does not import, have to check jupyter kernal
'''import sys

sys.executable
sys.path'''

['/private/var/folders/sb/jlzxdkbn2fvcmpk9q0m6hbdm0000gn/T/spark-8cafa602-e458-440e-b50a-34efa1f48d6b/userFiles-f5737e6e-96f1-4a73-bbf9-f49c8ccdd492',
 '',
 '/Users/shhudspeth/sharpest_minds/book_rec_sys',
 '/usr/local/Cellar/apache-spark/2.4.4/libexec/python/lib/py4j-0.10.7-src.zip',
 '/usr/local/Cellar/apache-spark/2.4.4/libexec/python',
 '/Users/shhudspeth/sharpest_minds/book_rec_sys',
 '/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python37.zip',
 '/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7',
 '/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/site-packages',
 '/usr/local/lib/python3.7/site-packages/IPython/extensions',
 '/Users/shhudspeth/.ipython']

In [19]:
!source ../../open_CV_env/bin/activate

In [22]:
import sys

sys.executable


'/usr/local/opt/python/bin/python3.7'

In [21]:
sys.path

['/private/var/folders/sb/jlzxdkbn2fvcmpk9q0m6hbdm0000gn/T/spark-8cafa602-e458-440e-b50a-34efa1f48d6b/userFiles-f5737e6e-96f1-4a73-bbf9-f49c8ccdd492',
 '',
 '/Users/shhudspeth/sharpest_minds/book_rec_sys',
 '/usr/local/Cellar/apache-spark/2.4.4/libexec/python/lib/py4j-0.10.7-src.zip',
 '/usr/local/Cellar/apache-spark/2.4.4/libexec/python',
 '/Users/shhudspeth/sharpest_minds/book_rec_sys',
 '/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python37.zip',
 '/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7',
 '/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/site-packages',
 '/usr/local/lib/python3.7/site-packages/IPython/extensions',
 '/Users/shhudspeth/.ipython']

In [14]:
from surprise import Reader, Dataset

# Define the format
reader = Reader(rating_scale=(1, 5))

# Load the data from the file using the reader format
data = Dataset.load_from_df(read_, reader=reader)

ModuleNotFoundError: No module named 'surprise'

In [10]:
from surprise.model_selection import cross_validate
from surprise import NormalPredictor


cross_validate(NormalPredictor(), data, cv=5)

ModuleNotFoundError: No module named 'surprise'

In [85]:
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import KFold


# define a cross-validation iterator
kf = KFold(n_splits=5)

algo = SVD()

for trainset, testset in kf.split(data):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error and Mean Average Error
    accuracy.rmse(predictions, verbose=True)
    accuracy.mae(predictions, verbose=True)

RMSE: 1.0099
MAE:  0.7756
RMSE: 1.0107
MAE:  0.7771
RMSE: 1.0092
MAE:  0.7752
RMSE: 1.0072
MAE:  0.7740
RMSE: 1.0122
MAE:  0.7764


In [ ]:
from collections import defaultdict

from surprise import SVD
from surprise import Dataset


def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


# First train an SVD algorithm on the movielens dataset.
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=10)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

In [ ]:
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    '''Return precision and recall at k metrics for each user.'''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls



kf = KFold(n_splits=5)
algo = SVD()

for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4)

    # Precision and recall can then be averaged over all users
    print(sum(prec for prec in precisions.values()) / len(precisions))
    print(sum(rec for rec in recalls.values()) / len(recalls))